In [2]:
import gurobipy as gp

from gurobipy import GRB
import numpy as np

# Farm Planning, problem 1
First, we create a LP problem for the first period--Janurary.

there's six decision variables: VEG1,VEG2,OIL1,OIL2,OIL3 and FINPROD. Our single period objective function is to minimize their costs and maximize their profit. 
$$P^t_{x_1} : \text{represents the price of X_1 at time t}$$


Our objective function is to maximize profit-- in other words, find the optimal comibination of all blended materials subtracted from the total profit of the finished product.

For a single time period t, the objective function is:
$$\text{max} \{ -P^t_{x_1}X_1 - P^t_{x_2} X_2 - P^t_{x_3}X_3 - P^t_{x_4}X_4 - P^t_{x_5}X_5 + 150 Y \} $$, where $Y$ is the amount of finished product.

We have to ensure that the hardness of the blend is kept within acceptable limits. 

$$ 8.8 X_1 + 6.1X_2 + 2.0X_3 + 4.2X_4 + 5.0X_5 - 3Y \leq 0$$ (lower bound hardness constraint)

$$ 8.8 X_1 + 6.1X_2 + 2.0X_3 + 4.2X_4 + 5.0X_5 - 6Y \leq 0$$ (upper bound hardness constraint)


We also need to ensure that there's continuity between the finished and final product, by adding a contuinity constraint:
$$X_1 + X_2 + X_3 + X_4 + X_5 - Y = 0$$

We also have the production constraints!

$$ X_1 + X_2 - 200 \leq 0 $$  
and
$$ X_3 + X_4 + X_5 \leq 0$$
We can't produce negative amounts of ingredients!

$$ X_i \geq 0 : i \in \{1,2,3,4,5\} $$

In [3]:
# Loading the data in first
prices = gp.tupledict({
	('January', 'VEG1'): 110,
	('January', 'VEG2'): 120,
	('January', 'OIL1'): 130,
	('January', 'OIL2'): 110,
	('January', 'OIL3'): 115,
	('February', 'VEG1'): 130,
	('February', 'VEG2'): 130,
	('February', 'OIL1'): 110,
	('February', 'OIL2'): 90,
	('February', 'OIL3'): 115,
	('March', 'VEG1'): 110,
	('March', 'VEG2'): 140,
	('March', 'OIL1'): 130,
	('March', 'OIL2'): 100,
	('March', 'OIL3'): 95,
	('April', 'VEG1'): 120,
	('April', 'VEG2'): 110,
	('April', 'OIL1'): 120,
	('April', 'OIL2'): 120,
	('April', 'OIL3'): 125,
	('May', 'VEG1'): 100,
	('May', 'VEG2'): 120,
	('May', 'OIL1'): 150,
	('May', 'OIL2'): 110,
	('May', 'OIL3'): 105,
	('June', 'VEG1'): 90,
	('June', 'VEG2'): 100,
	('June', 'OIL1'): 140,
	('June', 'OIL2'): 80,
	('June', 'OIL3'): 135
})

hardness_vals = gp.tupledict([('VEG1',8.8),('VEG2',6.1),('OIL1',2.0),('OIL2',4.2),('OIL3',5.0)])

decision_names = ['VEG1','VEG2','OIL1','OIL2','OIL3','FINPROD']

In [4]:
#building model for janurary.
current_month = 'January'
try:
    m = gp.Model()
    #add decision vars
    d_vars = m.addVars(decision_names,vtype=GRB.CONTINUOUS,name='raw_ingredients')
    #add objective function
    m.setObjective( (-1)*gp.quicksum(
        prices[(current_month,prod)] * d_vars[prod] 
                                     for prod in decision_names[0:-1] ) + 150 *d_vars['FINPROD'],
                   GRB.MAXIMIZE )
    #add hardness constraints
    hard_up = m.addConstr( gp.quicksum(hardness_vals[prod]*d_vars[prod] for prod in decision_names[0:-1])+ -6*d_vars['FINPROD'] <= 0)
    hard_low = m.addConstr( gp.quicksum(hardness_vals[prod]*d_vars[prod] for prod in decision_names[0:-1])+ -3*d_vars['FINPROD'] <= 0)
    
    #add continuity constraints
    continuity_constr = m.addConstr( gp.quicksum(d_vars[prod] for prod in decision_names[0:-1]) - d_vars['FINPROD'] == 0)
    
    veg_production_constr = m.addConstr(d_vars["VEG1"] + d_vars['VEG2'] - 200 <= 0)
    non_veg_production_constr = m.addConstr(d_vars['OIL1'] + d_vars['OIL2'] + d_vars['OIL3'] - 250<=0)

    positivityConstr = {}
    for i in decision_names:
        positivityConstr[i] = m.addConstr(d_vars[i] >= 0)
    
    
    
    m.update()
    m.optimize()
    
    findResults = m.X
    for i in range(len(findResults)):
        print(f'optimal soln has {findResults[i]} of {decision_names[i]}')
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))



Using license file /opt/gurobi901/gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (linux64)
Optimize a model with 11 rows, 6 columns and 29 nonzeros
Model fingerprint: 0xa9052414
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 2e+02]
Presolve removed 7 rows and 1 columns
Presolve time: 0.02s
Presolved: 4 rows, 5 columns, 15 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8000000e+04   9.812500e+01   0.000000e+00      0s
       3    7.4193548e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds
Optimal objective  7.419354839e+03
optimal soln has 0.0 of VEG1
optimal soln has 80.64516129032259 of VEG2
optimal soln has 250.0 of OIL1
optimal soln has 0.0 of OIL2
optimal soln has 0.0 of OIL3
optimal soln has 330.64516129032256 of FINPROD


It looks like we built an easy model for any month, but we should actually solve the problem, setting up a six-month plan.

To do that, we'll have to create month-specific decision variables, tie them in together, and make a really big block model!

To account for time, we decompose our decision variables $X_i$ into three:
1. $ X^{(\alpha) (t)}_i$ : represents the amount ingredient i is bought at month $t$  
2. $ X^{(\beta) (t)}_i$ : represents the amount ingredient i is stored after production on month $t$  
3. $ X^{(\delta) (t)}_i$ : represents the amount ingredient i is used during production of a final product on month $t$  


our objective function will change to:
$$max \sum_{\forall t} (-P^t_{x_1}X^{(\alpha) (t)}_1 - P^t_{x_2} X^{(\alpha) (t)}_2 - P^t_{x_3}X^{(\alpha) (t)}_3 - P^t_{x_4}X^{(\alpha) (t)}_4 - P^t_{x_5}X^{(\alpha) (t)}_5 + 150 Y^{(t)}) - \sum_{\forall t} 5(X^{(\beta) (t)}_1+X^{(\beta) (t)}_2+X^{(\beta) (t)}_3+X^{(\beta) (t)}_4+X^{(\beta) (t)}_5) $$

As this $-\sum_{\forall t} 5(X^{(\beta) (t)}_1+X^{(\beta) (t)}_2+X^{(\beta) (t)}_3+X^{(\beta) (t)}_4+X^{(\beta) (t)}_5$ clause represents the cost of having raw ingredients in storage, impacting the final objective. 

In temporalizing $X_i$, we have to represent the product of $Y^{(t)}$ with an affine constraint for each $t$:
$$Y^{(t)} - ( X^{(\delta) (t)}_1 + X^{(\delta) (t)}_2 + X^{(\delta) (t)}_3 + X^{(\delta) (t)}_4 + X^{(\delta) (t)}_5) = 0$$

We also have to temporalize the hardness constraints, using the production  $ X^{(\delta) (t)}_i$, for each $t$:
$$ 8.8X^{(\delta) (t)}_1 + 6.1 X^{(\delta) (t)}_2 + 2.0 X^{(\delta) (t)}_3 + 4.2 X^{(\delta) (t)}_4 + 5.0X^{(\delta) (t)}_5 - 3 Y^{(t) }\leq 0$$
$$ 8.8X^{(\delta) (t)}_1 + 6.1 X^{(\delta) (t)}_2 + 2.0 X^{(\delta) (t)}_3 + 4.2 X^{(\delta) (t)}_4 + 5.0X^{(\delta) (t)}_5 - 6 Y^{(t) }\leq 0$$

We also know that you cannot store more than 1000 tonnes of each oil per month, creating a bunch of inequality constraint:
$$X^{(\beta) (t)}_1+X^{(\beta) (t)}_2+X^{(\beta) (t)}_3+X^{(\beta) (t)}_4+X^{(\beta) (t)}_5  \leq 0$$

There's a conceptually interesting representation between treating the amount of $X_i$ is in storage, $X^{(\beta) (t)}_i$, as a decision variable which is mainly determined by a recursive constraint. We know that for t=0, all $X^{(\beta) (t)}_i = 500$.

For any $t > 0$, $X^{(\beta) (t)}_i$ is defined by the following constraint:
$$X^{(\beta) (t)}_i = X^{(\alpha) (t)}_i + X^{(\beta) (t-1)}_i - X^{(\delta) (t)}_i $$. In english, the amount of $X_i$ stored in month $t$ is the amount $X_i$ bought plus the amount $X_i$ we already had in storage, minus the amount we used up to make some final product. 

To enforce that we have 500 tonnes of each oil in June, we set the following constraint when t = 6:
$$X^{(\beta) (t=6)}_i - 500 = 0 $$


In [27]:
#first, make the month decision variables
months = ['January','February','March','April', 'May', 'June'  ]
month_decision_var_names = {}

for month in months:
    month_decision_var_names[month] = []

    for product in decision_names:
        if product == 'FINPROD':
            month_decision_var_names[month].append(product +'_' +month)
        else:
            month_decision_var_names[month].append(product + '_STORE_'+ month)
            month_decision_var_names[month].extend([ product + "_USED_" + month])
            month_decision_var_names[month].extend([product + '_BOUGHT_'+ month])
#month_decision_var_names #uncomment to check if correct



In [28]:
month_decision_var_names['January']

['VEG1_STORE_January',
 'VEG1_USED_January',
 'VEG2_STORE_January',
 'VEG2_USED_January',
 'OIL1_STORE_January',
 'OIL1_USED_January',
 'OIL2_STORE_January',
 'OIL2_USED_January',
 'OIL3_STORE_January',
 'OIL3_USED_January',
 'FINPROD_January']

In [22]:
try:
    m = gp.Model()
    #initialize block model by adding time-sensitive variables.
    month_decision_variables = {}
    #add decision variables for the month
    for month in months:
        month_decision_variables[month] = m.addVars(month_decision_var_names[month], vtype=GRB.CONTINUOUS)
        
    #print(month_decision_variables['January']) #sanity check
    
    
    #add general constraints (representing storage)
    
    #add objective function.
    
    
    #add decision vars
    d_vars = m.addVars(decision_names,vtype=GRB.CONTINUOUS,name='raw_ingredients')
    #add objective function
    m.setObjective( (-1)*gp.quicksum(
        prices[(current_month,prod)] * d_vars[prod] 
                                     for prod in decision_names[0:-1] ) + 150 *d_vars['FINPROD'],
                   GRB.MAXIMIZE )
    #add hardness constraints
    hard_up = m.addConstr( gp.quicksum(hardness_vals[prod]*d_vars[prod] for prod in decision_names[0:-1])+
                          -6*d_vars['FINPROD'] <= 0)
    hard_low = m.addConstr( gp.quicksum(hardness_vals[prod]*d_vars[prod] for prod in decision_names[0:-1])+
                           -3*d_vars['FINPROD'] <= 0)
    
    #add continuity constraints
    continuity_constr = m.addConstr( gp.quicksum(d_vars[prod] for prod in decision_names[0:-1]) - d_vars['FINPROD'] == 0)
    
    veg_production_constr = m.addConstr(d_vars["VEG1"] + d_vars['VEG2'] - 200 <= 0)
    non_veg_production_constr = m.addConstr(d_vars['OIL1'] + d_vars['OIL2'] + d_vars['OIL3'] - 250<=0)

    positivityConstr = {}
    for i in decision_names:
        positivityConstr[i] = m.addConstr(d_vars[i] >= 0)
    
    
    
    m.update()
    m.optimize()
    
    findResults = m.X
    for i in range(len(findResults)):
        print(f'optimal soln has {findResults[i]} of {decision_names[i]}')
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

    

{'VEG1_USED_January': <gurobi.Var *Awaiting Model Update*>, 'VEG1_STORE_January': <gurobi.Var *Awaiting Model Update*>, 'VEG2_USED_January': <gurobi.Var *Awaiting Model Update*>, 'VEG2_STORE_January': <gurobi.Var *Awaiting Model Update*>, 'OIL1_USED_January': <gurobi.Var *Awaiting Model Update*>, 'OIL1_STORE_January': <gurobi.Var *Awaiting Model Update*>, 'OIL2_USED_January': <gurobi.Var *Awaiting Model Update*>, 'OIL2_STORE_January': <gurobi.Var *Awaiting Model Update*>, 'OIL3_USED_January': <gurobi.Var *Awaiting Model Update*>, 'OIL3_STORE_January': <gurobi.Var *Awaiting Model Update*>, 'FINPROD_January': <gurobi.Var *Awaiting Model Update*>}
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (linux64)
Optimize a model with 11 rows, 72 columns and 29 nonzeros
Model fingerprint: 0x1f1e9e20
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 2e+02]
Presolve removed 7 rows and 67 columns
Pre

IndexError: list index out of range

In [ ]:
exampleTupleDict = gp.tupledict([('jan', (3,3))])
exampleTupleDict